In [2]:
import numpy as np

#csr from scartch
#value -> stores all non zero values in row major order
#col_indices -> stores the column index for each non-zero value
#row_ptr -> stores the index ranges for each row in the other arrays

class CSR_matrix:
    def __init__(self, dense_matrix):
        self.rows= len(dense_matrix)
        self.cols = len(dense_matrix[0]) if self.rows>0 else 0
        self.values= []
        self.col_indices = []
        self.row_ptr = [0]

        for row in dense_matrix:
            #number of zero in row
            nnz_in_row = 0
            for col_idx, value in enumerate(row):
                if value!=0:
                    self.values.append(value)
                    self.col_indices.append(col_idx)
                    nnz_in_row+=1
            self.row_ptr.append(self.row_ptr[-1]+nnz_in_row)

    def get_rows(self, row_idx):
        start = self.row_ptr[row_idx]
        end = self.row_ptr[row_idx+1]
        row = [0]*self.cols
        for i in range(start, end):
            col = self.col_indices[i]
            row[col] = self.values[i]
        return row
    
    def to_dense(self):
        
        dense =[[0]*self.cols for _ in range(self.rows)]
        for row_idx in range(self.rows):
            
        print(dense)



dense_matrix=[[1, 0, 2],
 [0, 0, 3],
 [4, 5, 0]]

csr = CSR_matrix(dense_matrix)
csr.to_dense()

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [3]:

dense_matrix=[[1, 0, 2],
 [0, 0, 3],
 [4, 5, 0]]



(0, 1)
(1, 2)
(2, 3)
